# 🤖 15:00–16:00 심화 1 – Agent & Tool & ReAct + Trace 분석

이 노트북은 `src/06_agent_tools.py` 내용을 **실습형 Jupyter**로 통합한 버전입니다.

**진행 순서**
1) 환경 변수/패키지 확인
2) Tool 정의(@tool) – 계산기/시간/간단 변환기
3) ReAct Agent 생성
4) create_react_agent로 자연어 질의 실행
5) LangSmith Trace 확인 포인트
6) 자유 실험 칸 (TODO)

> ⚠️ 실행 전 `.env`에 `OPENAI_API_KEY`, `LANGCHAIN_API_KEY`, `LANGCHAIN_TRACING_V2=true`, `LANGCHAIN_PROJECT=gccare-rag-workshop` 가 설정되어 있어야 합니다.

## 1) 환경 변수/패키지 확인

In [3]:
from pathlib import Path
import os
from dotenv import load_dotenv

BASE_DIR = Path.cwd().resolve().parents[0]
ENV_PATH = BASE_DIR / ".env"
load_dotenv(ENV_PATH)

print("BASE_DIR:", BASE_DIR)
print("OPENAI_API_KEY set?:", bool(os.getenv("OPENAI_API_KEY")))
print("LANGCHAIN_API_KEY set?:", bool(os.getenv("LANGCHAIN_API_KEY")))
print("LANGCHAIN_TRACING_V2:", os.getenv("LANGCHAIN_TRACING_V2"))
print("LANGCHAIN_PROJECT:", os.getenv("LANGCHAIN_PROJECT"))


BASE_DIR: /Users/sjcha/Documents/6. 개인작업 관리(smilechacha)/@모두의연구소/@모두의연구소 - 04 - GC케어&유비케어/202511_gccare/gccare-langchain-rag-ai
OPENAI_API_KEY set?: True
LANGCHAIN_API_KEY set?: True
LANGCHAIN_TRACING_V2: true
LANGCHAIN_PROJECT: gccare-rag-workshop


## 2) Tool 정의 (@tool)

- `@tool` 데코레이터로 **LLM이 사용할 수 있는 함수**를 등록합니다.
- Docstring(설명)과 타입 힌트가 **에이전트의 올바른 사용**에 큰 도움을 줍니다.

In [4]:
from datetime import datetime, timezone, timedelta
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """두 정수 a와 b의 합을 반환합니다."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """두 정수 a와 b의 곱을 반환합니다."""
    return a * b

@tool
def to_kst(iso_dt: str) -> str:
    """UTC ISO8601 날짜문자열(예:'2025-11-13T06:00:00Z')을 한국시간(KST, UTC+9)으로 변환하여 반환합니다."""
    dt = datetime.fromisoformat(iso_dt.replace('Z', '+00:00'))
    kst = dt.astimezone(timezone(timedelta(hours=9)))
    return kst.strftime('%Y-%m-%d %H:%M:%S KST')


## 3) ReAct Agent 생성 (LangChain Hub 프롬프트)

- `hwchase17/react` 템플릿을 LangChain Hub에서 가져옵니다.
- `create_react_agent` + `AgentExecutor` 로 에이전트를 구성합니다.

In [5]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

llm = ChatOpenAI(model="gpt-4o-mini")
tools = [add, multiply, to_kst]

SYSTEM_PROMPT = (
    "너는 계산 및 시간 변환 도구를 활용해 단계적으로 문제를 푸는 조교다. "
    "필요하면 도구를 사용하고, 최종적으로 간결한 한국어 답변을 제시하라. "
    "도구 입력은 JSON 형식을 유지하라."
)

# ---- 버전 호환 생성기 ----
def build_agent():
    # 우선 v2 + state_modifier 시도
    try:
        return create_react_agent(
            model=llm,
            tools=tools,
            state_modifier=SYSTEM_PROMPT,
            version="v2",          # 일부 버전은 version을 요구
        )
    except TypeError:
        pass

    # v1 + messages_modifier 시도
    try:
        return create_react_agent(
            model=llm,
            tools=tools,
            messages_modifier=SYSTEM_PROMPT,
            version="v1",
        )
    except TypeError:
        pass

    # 마지막 폴백: modifier 없이 생성 → 호출 시 SystemMessage를 함께 넘김
    return create_react_agent(
        model=llm,
        tools=tools,
        # version 인자도 어떤 설치본은 필수일 수 있음 — 필요하면 한 번 더 시도
    )

agent = build_agent()

# === 단건 실행 (modifier를 못 넣은 폴백 케이스도 커버) ===
def run_once(question: str):
    try:
        # modifier가 먹힌 버전
        res = agent.invoke(
            {"messages": [HumanMessage(content=question)]},
            config={"recursion_limit": 8},
        )
    except Exception:
        # 폴백: 호출 시 SystemMessage를 같이 전달
        res = agent.invoke(
            {"messages": [SystemMessage(content=SYSTEM_PROMPT),
                          HumanMessage(content=question)]},
            config={"recursion_limit": 8},
        )
    print(res["messages"][-1].content)

run_once("10과 32를 더한 다음 2배로 해줘.")


/var/folders/jc/4z1x4d2x08v3pks_cvns5kzr0000gn/T/ipykernel_28680/3309104646.py:19: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  return create_react_agent(
/var/folders/jc/4z1x4d2x08v3pks_cvns5kzr0000gn/T/ipykernel_28680/3309104646.py:30: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  return create_react_agent(
/var/folders/jc/4z1x4d2x08v3pks_cvns5kzr0000gn/T/ipykernel_28680/3309104646.py:40: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  return create_react_agent(
/Users/sjcha/Documents/6. 개

10과 32를 더한 결과는 42입니다. 

그 후, 이 결과를 2배로 하면 84가 됩니다.


## 4) Agent 실행 – 자연어로 Tool 사용 지시

아래 예시 질의를 실행해 보고, 어떤 Tool을 선택/호출했는지 확인합니다.

### 질문별 독립 실행(Stateless)

In [6]:
questions = [
    "10과 32를 더한 다음, 그 결과를 2배로 만든 값을 알려줘.",
    "'2025-11-13T06:00:00Z' 를 한국시간으로 바꿔줘.",
    "7과 8을 곱한 뒤, 10을 더해줘. 필요한 계산은 도구를 사용해.",
]

for q in questions:
    print("\n==============================")
    print("질문:", q)
    res = agent.invoke(
        {"messages": [HumanMessage(content=q)]},
        config={"recursion_limit": 8},  # 도구 호출 반복 제한
    )
    # 응답 추출
    msgs = res.get("messages", [])
    answer = msgs[-1].content if msgs else res
    print("\n최종 답변:", answer)


질문: 10과 32를 더한 다음, 그 결과를 2배로 만든 값을 알려줘.

최종 답변: 10과 32를 더한 결과는 42이고, 이 값을 2배로 만든 값은 84입니다.

질문: '2025-11-13T06:00:00Z' 를 한국시간으로 바꿔줘.

최종 답변: '2025-11-13T06:00:00Z'는 한국시간으로 '2025-11-13 15:00:00 KST'입니다.

질문: 7과 8을 곱한 뒤, 10을 더해줘. 필요한 계산은 도구를 사용해.

최종 답변: 7과 8을 곱한 결과는 56이며, 여기에 10을 더한 결과는 66입니다.


### 대화 맥락 유지 (Stateful)

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

history = []  # 세션 히스토리

def ask(q: str):
    print("\n==============================")
    print("질문:", q)
    history.append(HumanMessage(content=q))
    out = agent.invoke({"messages": history}, config={"recursion_limit": 8})
    # LangGraph state에는 전체 히스토리가 들어있으므로, 최신으로 교체
    history[:] = out["messages"]
    print("\n최종 답변:", history[-1].content)

for q in [
    "10과 32를 더한 다음, 그 결과를 2배로 만든 값을 알려줘.",
    "'2025-11-13T06:00:00Z' 를 한국시간으로 바꿔줘.",
    "7과 8을 곱한 뒤, 10을 더해줘. 필요한 계산은 도구를 사용해.",
]:
    ask(q)



질문: 10과 32를 더한 다음, 그 결과를 2배로 만든 값을 알려줘.

최종 답변: 10과 32를 더한 결과는 42입니다. 그 값을 2배로 만들면 84가 됩니다.

질문: '2025-11-13T06:00:00Z' 를 한국시간으로 바꿔줘.

최종 답변: '2025-11-13T06:00:00Z'는 한국시간(KST)으로 '2025-11-13 15:00:00'입니다.

질문: 7과 8을 곱한 뒤, 10을 더해줘. 필요한 계산은 도구를 사용해.

최종 답변: 7과 8을 곱한 결과는 56입니다. 여기에 10을 더하면 66이 됩니다.


## 5) LangSmith Trace 확인 포인트

- Run 트리에서 확인할 항목
  - 최상단: `agent` 호출
  - 내부: `LLM` 호출(Thought 생성), `Tool` 호출(Action/Observation), 다시 `LLM` 호출(최종 답변 작성)
- 체크리스트
  - Thought가 정확히 어떤 이유로 특정 Tool을 선택했는가?
  - Tool 입력/출력이 의도한 타입/형식으로 오갔는가?
  - 같은 문제를 더 적은 호출로 해결하는 Prompt 개선 여지가 있는가?


## 6) 자유 실험 칸 (TODO)

아이디어 예시:
- 새로운 Tool을 1~2개 더 정의하고 Agent에 등록해 보기
- 다른 Hub 프롬프트를 써보기 (예: react-json 등)
- 실패/에러 상황을 유도하고 에러 핸들링 전략 세워보기

In [ ]:
# TODO: 새 Tool 정의해서 등록해보기


In [ ]:
# TODO: 다른 Hub 프롬프트를 써보기 (예: react-json 등)


In [ ]:
# TODO: 실패/에러 상황을 유도하고 에러 핸들링 전략 세워보기


---
## 📌 추가 개인 실습 – EMR 도메인용 RAG + 계산/요약 Agent

**목표**
- 기존 EMR RAG 체인을 **툴로 감싼 후**, ReAct Agent가 RAG + 계산 Tool을 조합해서 사용하는 흐름을 만들어 본다.
- Thought / Action / Observation을 LangSmith Trace에서 읽는 연습을 한다.

**준비**
- 이전 시간에 만든 EMR RAG 체인 또는 Retriever 함수
- 이 노트북의 `@tool` 예제, `create_react_agent`, `AgentExecutor` 코드



---
### TODO 1. EMR 전용 Tool 정의하기

아래와 같은 Tool 2~3개를 직접 정의해 보세요.

1. `@tool def emr_rag(query: str) -> str:`  
   - 내부에서 EMR RAG 체인을 호출하고, **짧은 한국어 요약 답변**만 문자열로 반환

2. `@tool def calc_claim_error_rate(total: int, error: int) -> float:`  
   - 청구 총 건수(`total`)와 오류 건수(`error`)를 받아 오류율(%)을 계산

3. (선택) `@tool def to_kst_now() -> str:`  
   - 현재 시간을 KST 기준 ISO 문자열로 반환

> 각 Tool의 docstring은 “언제, 어떤 용도로 쓰는 함수인지”를 자연어로 최대한 구체적으로 적어보세요.


In [ ]:
# TODO 1: emr_rag, calc_claim_error_rate, to_kst_now 등의 Tool을 정의해 보세요.



---
### TODO 2. ReAct Agent로 복합 질의 처리해보기

- LangChain Hub의 `hwchase17/react` 프롬프트를 사용해 Agent를 구성하고,
  - tools = `[emr_rag, calc_claim_error_rate]` (및 선택 Tool) 를 넘겨주세요.
  - `max_iterations`를 4~5 정도로 제한해 무한 루프를 방지합니다.

- 아래와 같은 질문을 던져 보고, Thought / Action / Observation 로그를 확인해 보세요.

  - *"지난달 외래 청구 1200건 중 45건이 오류였어. 오류율을 계산하고, EMR 매뉴얼을 참고해서 개선 팁 2가지만 알려줘."*  
  - *"검사 오더 입력 실수를 줄이기 위한 프로세스를 문서 기반으로 제안해줘. 필요하면 오류율 계산 툴도 활용해."*

> LangSmith Trace에서 Agent → Tool 호출 순서를 스크린샷으로 남겨두면, 이후 회고/공유 자료에 활용하기 좋습니다.


In [ ]:
# TODO 2: 위 Tool들을 사용해 ReAct Agent를 구성하고, 예시 질문을 던져보세요.
